## imports

In [12]:
from utils import load_df
import pandas as pd
from tensorflow.keras import datasets, layers, models, losses

## Load Data

In [10]:
df = load_df("combined_weather_data_from_2009_to_present.pkl")
print(df.shape)

(104520, 26)


## Prepare data

#### Input data :
les points de données des instants `t-n` à `t`
- débit horaire [int]
- Taux d'occupation [float]
- météo : température, couverture nuageuse, ...

#### Output data :
for each arc (x3) instant `t+1`:
- débit horaire [int]
- Taux d'occupation [float]

## First model

In [ ]:
def model_1(input_shape, output_size=6, show_summary=False):
    model = models.Sequential()

    model.add(layers.Conv2D(128, kernel_size=(3, 3),
                        input_shape=input_shape, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(3))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(output_size, activation="linear"))
    
    if show_summary:
        model.summary()
        
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    return model